In [17]:
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext

import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import Window

import json
import numpy as np
import pandas as pd
from datetime import datetime
import re

sc = SparkContext.getOrCreate()
spark = SQLContext(sc)


/home/dulat/.local/lib/python3.8/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [18]:
"""
назначаем отчетный месяц
"""
date = "2022-08-01"
formatted_date = pd.to_datetime(date, format="%Y-%m-%d")
input_date = formatted_date.strftime("%Y%m")


"""
считываем исходные данные
"""
df_cdr_SERIAL_NUM_daily = spark.read.csv("../data/first_dataset.csv", sep=';', header=True)
df_cdr_SERIAL_NUM_daily = (
    df_cdr_SERIAL_NUM_daily.select("*")
    .withColumn("ID", (F.col("ID").cast("string")))
    .withColumn("SERIAL_NUM", (F.col("SERIAL_NUM").cast("string")))
)
df_cdr_SERIAL_NUM_daily_replica = df_cdr_SERIAL_NUM_daily
df_cdr_SERIAL_NUM_daily.registerTempTable("df_cdr_SERIAL_NUM_daily")

sql_agg_num = """select ID, SERIAL_NUM, count(SERIAL_NUM) as count_SERIAL_NUM 
                    from df_cdr_SERIAL_NUM_daily
                    group by ID, SERIAL_NUM
            """

sql_agg_count = 0
sql_agg_count = spark.sql(sql_agg_num)

sql_agg_count.registerTempTable("num_agg_count")

"""
Скрипт для рассчета наиболее часто встречающегося SERIAL_NUM за месяц у абонента
"""

sql_agg_r = """
        select * ,
             ROW_NUMBER() OVER (PARTITION BY ID ORDER BY ID, count_SERIAL_NUM DESC) AS RowNum
                from num_agg_count 

          """

sql_agg_rank = 0
sql_agg_rank = spark.sql(sql_agg_r)

sql_agg_rank.registerTempTable("num_agg_rank")

sql_agg_r_1 = """
                select ID, SERIAL_NUM as MAJOR_SERIAL_NUM_1M
                    from num_agg_rank 
                    where RowNum =1
          """
sql_agg_rank_1 = 0
sql_agg_rank_1 = spark.sql(sql_agg_r_1)

"""
Агрегация исходных данных(трафиковые, количество уникальных SERIAL_NUM в месяц у абонента, первый и последний SERIAL_NUM в отчетном периоде)
"""
df_cdr_SERIAL_NUM_daily_replica = df_cdr_SERIAL_NUM_daily_replica.groupBy(["ID"]).agg(
    F.sum("CALL_DURATION_MIN").alias("CALL_DURATION_MIN_MONTHLY"),
    F.sum("NUM_OF_CALLS").alias("NUM_OF_CALLS_MONTHLY"),
    F.sum("TRAFFIC_LTE").alias("TRAFFIC_LTE_MONTHLY"),
    F.countDistinct("SERIAL_NUM").alias("SERIAL_NUM_ON_SUBS_MONTHLY"),
    F.first(df_cdr_SERIAL_NUM_daily_replica["SERIAL_NUM"]).alias("FIRST_SERIAL_NUM_MONTHLY"),
    F.last(df_cdr_SERIAL_NUM_daily_replica["SERIAL_NUM"]).alias("LAST_SERIAL_NUM_MONTHLY"),
)


"""
Джойним результаты аггрегации с таблицей с наиболее часто встречающимся SERIAL_NUM за месяц
"""
df_cdr_SERIAL_NUM_daily_replica = df_cdr_SERIAL_NUM_daily_replica.withColumn(
    "MONTH_KEY", F.lit(input_date)
)
df_combined = df_cdr_SERIAL_NUM_daily_replica.join(
    sql_agg_rank_1, on=["ID"], how="left"
)

# """
# считываем планы запроса
# """
# df_combined.explain(True)


"""
считываем только физический план запроса
"""
df_combined.explain("formatted")

/home/dulat/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


== Physical Plan ==
AdaptiveSparkPlan (24)
+- Project (23)
   +- BroadcastHashJoin LeftOuter BuildRight (22)
      :- SortAggregate (10)
      :  +- Sort (9)
      :     +- Exchange (8)
      :        +- SortAggregate (7)
      :           +- SortAggregate (6)
      :              +- Sort (5)
      :                 +- Exchange (4)
      :                    +- SortAggregate (3)
      :                       +- Sort (2)
      :                          +- Scan csv  (1)
      +- BroadcastExchange (21)
         +- Project (20)
            +- Filter (19)
               +- Window (18)
                  +- Sort (17)
                     +- Exchange (16)
                        +- HashAggregate (15)
                           +- Exchange (14)
                              +- HashAggregate (13)
                                 +- Filter (12)
                                    +- Scan csv  (11)


(1) Scan csv 
Output [5]: [ID#1078, SERIAL_NUM#1079, CALL_DURATION_MIN#1080, NUM_OF_CALLS#1081, T